## 단어 단위 RNN - 임베딩 층 사용

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
sentence = "Repeat is the best medicine for memory".split()

In [3]:
# 단어집합 생성
vocab = list(set(sentence))
print(vocab)

['is', 'best', 'memory', 'the', 'Repeat', 'for', 'medicine']


In [5]:
# 단어에 고유 정수 부여, 모르는 단어를 의미하는 UNK 토큰도 추가
word2index = {tkn: i for i, tkn in enumerate(vocab, 1)}
word2index['<unk>'] = 0
print(word2index)

{'is': 1, 'best': 2, 'memory': 3, 'the': 4, 'Repeat': 5, 'for': 6, 'medicine': 7, '<unk>': 0}


In [6]:
print(word2index['memory'])

3


In [7]:
# 수치화된 데이터를 단어로 바꾸기 위한 사전
index2word = {v: k for k, v in word2index.items()}
print(index2word)

{1: 'is', 2: 'best', 3: 'memory', 4: 'the', 5: 'Repeat', 6: 'for', 7: 'medicine', 0: '<unk>'}


In [9]:
print(index2word[3])

memory


In [10]:
def build_data(sentence, word2index):
    encoded = [word2index[token] for token in sentence] # 각 문자를 정수로 변환. 
    input_seq, label_seq = encoded[:-1], encoded[1:] # 입력 시퀀스와 레이블 시퀀스를 분리
    input_seq = torch.LongTensor(input_seq).unsqueeze(0) # 배치 차원 추가
    label_seq = torch.LongTensor(label_seq).unsqueeze(0) # 배치 차원 추가
    return input_seq, label_seq

In [11]:
X, Y = build_data(sentence, word2index)

In [12]:
print(X)
print(Y)

tensor([[5, 1, 4, 2, 7, 6]])
tensor([[1, 4, 2, 7, 6, 3]])


In [13]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, # 워드 임베딩
                                            embedding_dim=input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size, # 입력 차원, 은닉 상태의 크기 정의
                                batch_first=batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size) # 출력은 원-핫 벡터의 크기를 가져야함. 또는 단어 집합의 크기만큼 가져야함.

    def forward(self, x):
        # 1. 임베딩 층
        # 크기변화: (배치 크기, 시퀀스 길이) => (배치 크기, 시퀀스 길이, 임베딩 차원)
        output = self.embedding_layer(x)
        # 2. RNN 층
        # 크기변화: (배치 크기, 시퀀스 길이, 임베딩 차원)
        # => output (배치 크기, 시퀀스 길이, 은닉층 크기), hidden (1, 배치 크기, 은닉층 크기)
        output, hidden = self.rnn_layer(output)
        # 3. 최종 출력층
        # 크기변화: (배치 크기, 시퀀스 길이, 은닉층 크기) => (배치 크기, 시퀀스 길이, 단어장 크기)
        output = self.linear(output)
        # 4. view를 통해서 배치 차원 제거
        # 크기변화: (배치 크기, 시퀀스 길이, 단어장 크기) => (배치 크기*시퀀스 길이, 단어장 크기)
        return output.view(-1, output.size(2))

In [14]:
# 하이퍼 파라미터
vocab_size = len(word2index)  # 단어장의 크기는 임베딩 층, 최종 출력층에 사용된다. <unk> 토큰을 크기에 포함한다.
input_size = 5  # 임베딩 된 차원의 크기 및 RNN 층 입력 차원의 크기
hidden_size = 20  # RNN의 은닉층 크기

In [15]:
# 모델 생성
model = Net(vocab_size, input_size, hidden_size, batch_first=True)
# 손실함수 정의
loss_function = nn.CrossEntropyLoss() # 소프트맥스 함수 포함이며 실제값은 원-핫 인코딩 안 해도 됨.
# 옵티마이저 정의
optimizer = optim.Adam(params=model.parameters())

In [17]:
# 임의로 예측해보기. 가중치는 전부 랜덤 초기화 된 상태이다.
output = model(X)
print(output)
print(output.shape)

tensor([[-0.0270, -0.1613,  0.0775, -0.0812, -0.2864, -0.0444,  0.0464, -0.1417],
        [-0.1239, -0.1917,  0.1607, -0.4938, -0.1650,  0.1276, -0.1272,  0.0777],
        [ 0.0720, -0.0191,  0.0336, -0.3742, -0.0138, -0.0013, -0.1384, -0.0394],
        [ 0.2119, -0.0887, -0.1776,  0.0267, -0.1967, -0.3027, -0.0254, -0.3144],
        [-0.1631, -0.0985,  0.1452, -0.3771, -0.2494,  0.0529, -0.2385,  0.0906],
        [ 0.0512, -0.1422,  0.0721, -0.2855, -0.1005,  0.0267, -0.0183, -0.1272]],
       grad_fn=<ViewBackward0>)
torch.Size([6, 8])


In [18]:
# 수치화된 데이터를 단어로 전환하는 함수
decode = lambda y: [index2word.get(x) for x in y]

In [19]:
# 훈련 시작
for step in range(201):
    # 경사 초기화
    optimizer.zero_grad()
    # 순방향 전파
    output = model(X)
    # 손실값 계산
    loss = loss_function(output, Y.view(-1))
    # 역방향 전파
    loss.backward()
    # 매개변수 업데이트
    optimizer.step()
    # 기록
    if step % 40 == 0:
        print("[{:02d}/201] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["Repeat"] + decode(pred)))
        print()

[01/201] 2.1948 
Repeat best best <unk> <unk> best best

[41/201] 1.7989 
Repeat memory best best memory best best

[81/201] 1.2306 
Repeat is the best medicine for memory

[121/201] 0.5785 
Repeat is the best medicine for memory

[161/201] 0.2585 
Repeat is the best medicine for memory

[201/201] 0.1350 
Repeat is the best medicine for memory

